<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Gradient Descent in Sklearn

_Authors: Kiefer Katovich (SF)_

---

Until now we've been using specific sklearn model classes to perform regression and classification such as `LinearRegression` and `LogisticRegression`. Unfortunately, while these methods work well on smaller datasets with relatively small numbers of columns, once you start getting into "Medium Data" these slow down to a crawl, and take up so much memory that fitting them becomes mind-numbingly slow (especially on a laptop).

Luckily, sklearn comes with  stochastic gradient descent solvers for regression and classification:
- `SGDRegressor`
- `SGDClassifier`

Due to its ability to minimize the loss function iteratively on smaller portions of the data, it avoids the intense slowdown other models suffer on large datasets.

> **Note:** The gradient descent solvers are very flexible and can fit a variety of different model types not covered here. I highly recommend reading their documentation in detail.

---

### SF assessor data

This lab uses data from the SF assessor's office on housing prices in San Francisco - it's already cleaned up.

You can see that the dataset has 250k rows. When expanding this with dummy-coded categorical columns it can become quite large. Be careful that you don't exceed the memory on your computer.


In [3]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import scipy.stats as stats

import patsy

import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

### 1. Load the data.

Examine the columns.

In [4]:
prop = pd.read_csv('../datasets/assessor_sample.csv')

In [5]:
prop.shape

(250000, 17)

In [6]:
prop.dtypes

baths               int64
beds                int64
lot_depth         float64
basement_area     float64
front_ft          float64
owner_pct         float64
rooms               int64
property_class     object
neighborhood       object
tax_rate          float64
volume              int64
sqft                int64
stories             int64
year_recorded       int64
year_built          int64
zone               object
value             float64
dtype: object

In [7]:
prop.isnull().sum()

baths             0
beds              0
lot_depth         0
basement_area     0
front_ft          0
owner_pct         0
rooms             0
property_class    0
neighborhood      0
tax_rate          0
volume            0
sqft              0
stories           0
year_recorded     0
year_built        0
zone              0
value             0
dtype: int64

In [8]:
prop.property_class.unique()

array(['D', 'Z', 'DBM', 'LZ', 'TH', 'ZBM'], dtype=object)

In [9]:
prop.neighborhood.unique()

array(['01E', '10G', '10C', '02C', '02A', '04S', '08F', '10H', '03A',
       '02F', '10B', '04T', '04J', '04M', '08E', '10F', '10E', '03G',
       '03H', '06D', '09E', '02B', '02D', '10D', '01C', '07B', '02E',
       '04C', '04H', '10A', '05A', '04F', '04E', '10J', '09A', '03B',
       '07C', '01A', '01G', '09G', '02G', '05C', '05F', '03J', '05K',
       '03E', '06A', '05D', '08C', '04G', '04R', '06B', '01D', '07A',
       '09C', '09F', '10K', '01F', '05G', '03C', '09H', '04B', '08G',
       '04N', '05B', '08A', '08H', '04P', '03F', '07D', '06E', '01B',
       '05M', '05E', '06C', '04D', '05H', '08B', '05J', '09D', '04A',
       '04K', '08D', '06F', '03D', '09B', '047', '08I'], dtype=object)

### 2. Sample down the data

Despite this already being a sample of the full assessor dataset, you should sample the data down further the sake of speed and your computers memory.

Use the `.sample()` function for pandas dataframes to subset this down to < 25000 rows. 

Sampling down large datasets is a common procedure. Finding the optimal parameters with larger subsets of the data may change the hyperparameters and the results, and will get you closer to the best coefficients, but the returns are marginal at a point.

In [10]:
prop_samp = prop.sample(n=25000)

### 3. Regression with stochastic gradient descent

Below I set up X, y data predicting value (housing price) from the remaining variables. There are ~75,000 rows, with 170 columns.


The `SGDRegressor` is very general and flexible, and can be customized with a variety of keyword arguments.

**Arguments**
- `loss`: `['squared_loss','huber', ...]`
    - The `'squared_loss'` loss corresponds to solving a regression with the least squares loss. This is what I expect you'll use, but there are other options. Huber loss is a "robust" regression loss.
- `penalty`: `['none','l1','l2','elasticnet']`
    - This defines the penalty on the regression that you would like to solve. The l1 and l2 are the Lasso and Ridge, while the elasticnet is the combination of them both.
- `alpha`
    - The regularization strength to be used with a chosen penalty. Same as in Lasso and Ridge.
- `l1_ratio`
    - The mix of the Lasso and Ridge penalties when elasticnet is chosen as the penalty.
- `n_iter`
    - The number of training "epochs" over the data. This is the number of passes that the gradient descent algorithm will make over the data to iteratively fit the weights (defaults to 5).

`SGDRegressor` is most often used in tandem with grid searching to find the optimal parameters for certain models. 

**It is up to you how you want to define the model. You should:**

1. Choose a target to estimate (this should be continuous).
- Select predictors to use.
- Standardize your predictor matrix.
- Build a stochastic gradient descent solver to fit your model. You will likely want to do some kind of gridsearch to find the optimal parameters for your model.
- Describe the model selected through gridsearch and compare the performance to baseline.
- Examine and interpret the coefficients.

In [11]:
f = 'value ~ '+' + '.join([c for c in prop_samp.columns if not c == 'value'])+' -1'
print f
y, X = patsy.dmatrices(f, data=prop_samp, return_type='dataframe')
y = y.values.ravel()

print y.shape, X.shape

value ~ baths + beds + lot_depth + basement_area + front_ft + owner_pct + rooms + property_class + neighborhood + tax_rate + volume + sqft + stories + year_recorded + year_built + zone -1
(25000,) (25000, 163)


In [12]:
# I am predicting the value of the house from the other variables.

In [29]:
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [14]:
# standardize the predictors
ss = StandardScaler()
Xs = ss.fit_transform(X)
print Xs.shape

(25000, 163)


In [30]:
# set up my gridsearch parameters:
sgd_params = {
    'loss':['squared_loss','huber'],
    'penalty':['l1','l2'],
    'alpha':np.logspace(-5,1,25)
}

sgd_reg = SGDRegressor()
sgd_reg_gs = GridSearchCV(sgd_reg, sgd_params, cv=5, verbose=False)

In [31]:
# SGD is pretty fast compared to other sklearn solvers - but can still
# take a good long while depending on the gridsearch and the size of
# the dataset.
sgd_reg_gs.fit(Xs, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'loss': ['squared_loss', 'huber'], 'alpha': array([  1.00000e-05,   1.77828e-05,   3.16228e-05,   5.62341e-05,
         1.00000e-04,   1.77828e-04,   3.16228e-04,   5.62341e-04,
         1.00000e-03,   1.77828e-03,   3.16228e-03,   5.62341e-03,
         1.00000e-...2341e-01,
         1.00000e+00,   1.77828e+00,   3.16228e+00,   5.62341e+00,
         1.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=False)

In [17]:
print sgd_reg_gs.best_params_
print sgd_reg_gs.best_score_
sgd_reg = sgd_reg_gs.best_estimator_

{'penalty': 'l2', 'alpha': 1.0, 'loss': 'squared_loss'}
0.197097050112


In [18]:
# We can see that the gradient descent got a .22 R2 using the ridge and squared loss.

In [19]:
value_coefs = pd.DataFrame({'coef':sgd_reg.coef_,
                            'mag':np.abs(sgd_reg.coef_),
                            'pred':X.columns})
value_coefs.sort_values('mag', ascending=False, inplace=True)
value_coefs.iloc[0:10, :]

,coef,mag,pred
143,30973.212863,30973.212863,zone[T.SACTO]
151,26780.222137,26780.222137,beds
47,23451.630577,23451.630577,neighborhood[T.05C]
159,21676.077188,21676.077188,sqft
155,-18560.858293,18560.858293,owner_pct
62,17215.526707,17215.526707,neighborhood[T.07A]
150,16389.914561,16389.914561,baths
52,14847.013586,14847.013586,neighborhood[T.05H]
127,14315.475303,14315.475303,zone[T.RH3]
158,-14245.125629,14245.125629,volume


### 4. Classification with stochastic gradient descent

The `SGDClassifier` is very similar to the `SGDRegressor`. The main difference is that the loss functions are changed to regression loss functions.

**Arguments**
- `loss`: `['log', ...]`
    - The `'log'` loss corresponds to solving a logistic regression classifier. This is what I expect you'll use, but there are many other options.
- `penalty`: `['none','l1','l2','elasticnet']`
    - This defines the penalty on the regression that you would like to solve. The l1 and l2 are the Lasso and Ridge, while the elasticnet is the combination of them both.
- `alpha`
    - The regularization strength to be used with a chosen penalty. Same as in Lasso and Ridge.
- `l1_ratio`
    - The mix of the Lasso and Ridge penalties when elasticnet is chosen as the penalty.
- `n_iter`
    - The number of training "epochs" over the data. This is the number of passes that the gradient descent algorithm will make over the data to iteratively fit the weights (defaults to 5).

Like `SGDRegressor`, `SGDClassifier` is most often used in tandem with grid searching to find the optimal parameters for certain models. 

**It is up to you how you want to define the model. You should:**

1. Choose a target to classify (you may need to engineer one from existing variables).
- Calculate the baseline accuracy.
- Select predictors to use.
- Standardize your predictor matrix.
- Build a stochastic gradient descent solver to fit your model. You will likely want to do some kind of gridsearch to find the optimal parameters for your model.
- Describe the model selected through gridsearch and compare the performance to baseline.
- Examine and interpret the coefficients.

In [20]:
prop_samp.columns

Index([u'baths', u'beds', u'lot_depth', u'basement_area', u'front_ft',
       u'owner_pct', u'rooms', u'property_class', u'neighborhood', u'tax_rate',
       u'volume', u'sqft', u'stories', u'year_recorded', u'year_built',
       u'zone', u'value'],
      dtype='object')

In [21]:
prop_samp.year_built.value_counts()

1941    1015
1940     897
1925     877
1926     776
1924     764
1927     698
1923     693
1939     643
1947     603
1948     514
1908     491
1928     478
1922     467
1946     466
1906     459
1950     437
1938     421
1907     417
1951     411
1931     375
1944     363
1910     349
1930     344
1936     344
1949     342
1929     333
1937     315
1942     313
1912     294
1955     260
        ... 
1984     114
1979     113
1993     111
1997     107
1933     102
1995     102
1985      92
1998      91
1918      84
1994      82
1976      77
1973      73
1966      67
1977      60
1970      54
1974      54
1902      52
1903      49
1967      46
1934      43
2000      43
1968      42
1971      32
1969      28
1999      20
1901      11
2001       5
2003       3
2002       1
2005       1
Name: year_built, dtype: int64

In [22]:
# lets see if we can predict if a house was built past 1980
prop_samp['built_past1980'] = prop_samp.year_built.map(lambda x: 1 if x >= 1980 else 0)

In [23]:
# make the target and calculate the baseline:
y = prop_samp.built_past1980.values
print 1. - np.mean(y)

0.89576


In [24]:
f = '''
~ baths + beds + lot_depth + basement_area + front_ft + owner_pct +
rooms + property_class + neighborhood + tax_rate + volume + sqft + stories +
zone + value -1
'''

In [25]:
X = patsy.dmatrix(f, data=prop_samp, return_type='dataframe')

Xs = ss.fit_transform(X)
print y.shape, Xs.shape

(25000,) (25000, 162)


In [26]:
sgd_cls_params = {
    'loss':['log'],
    'penalty':['l1','l2'],
    'alpha':np.logspace(-5,2,50)
}

sgd_cls = SGDClassifier()
sgd_cls_gs = GridSearchCV(sgd_cls, sgd_cls_params, cv=5, verbose=1)

In [27]:
sgd_cls_gs.fit(Xs, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   52.2s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'loss': ['log'], 'alpha': array([  1.00000e-05,   1.38950e-05,   1.93070e-05,   2.68270e-05,
         3.72759e-05,   5.17947e-05,   7.19686e-05,   1.00000e-04,
         1.38950e-04,   1.93070e-04,   2.68270e-04,   3.72759e-04,
         5.17947e-04,   7.19686e-04,...    1.93070e+01,   2.68270e+01,   3.72759e+01,   5.17947e+01,
         7.19686e+01,   1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [28]:
print sgd_cls_gs.best_params_
print sgd_cls_gs.best_score_
sgd_cls = sgd_cls_gs.best_estimator_

{'penalty': 'l1', 'alpha': 0.00019306977288832496, 'loss': 'log'}
0.96236
